In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


## Carga de datos

### Datos 2023-2024

In [5]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data/Totalizadores Planta de Cerveza 2023_2024.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs_2023_2024 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2023_2024[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  12010       125   
1                       Metas     48        57   
2      Consolidado Produccion  12011        19   
3    Totalizadores Produccion  12009        41   
4              Consolidado EE  12011        24   
5       Totalizadores Energia  12009        54   
6            Consolidado Agua  12011        24   
7          Totalizadores Agua  12009        44   
8        Consolidado GasVapor  12011        20   
9   Totalizadores Gas y Vapor  12009        24   
10           Consolidado Aire  12011        14   
11         Totalizadores Aire  12009        12   
12          Totalizadores CO2  12009         9   
13    Totalizadores Efluentes  12009         9   
14       Totalizadores Glicol  12009         8   
15            Seguimiento Dia  12009         4   
16                   Auxiliar  12011        38   
17          Kw Frio  Hl Mosto  12372        15   

             

#### Código para ver una hoja específica

In [6]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2023_2024:

    df_hoja = dfs_2023_2024[nombre_hoja_para_ver]
    
    print(f"Mostrando la hoja: {nombre_hoja_para_ver}")
    print(f"Dimensiones: {df_hoja.shape[0]} filas x {df_hoja.shape[1]} columnas")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(df_hoja.head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(df_hoja.tail(3).to_string())
else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2023_2024.keys()))

Mostrando la hoja: Consolidado KPI
Dimensiones: 12010 filas x 125 columnas

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / H

#### Código para detectar días con horas faltantes

In [7]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2023_2024.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_set = set(todos_los_dias) - set(dias_con_23_59)
        dias_sin_23_59_lista = sorted(list(dias_sin_23_59_set))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_con_23_59 = sorted(list( set(dias_con_horas_faltantes_sr.index) - dias_sin_23_59_set ))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_con_23_59)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59 dias_con_horas_faltantes
hoja                                                                                                                            
Consolidado KPI            2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)   2023-01-17, 2024-07-01
Metas                      2021-01-01  2024-12-01                                 N/A (Hoja no horaria)    N/A (Hoja no horaria)
Consolidado Produccion     2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)   2023-01-17, 2024-07-01
Totalizadores Produccion   2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)   2023-01-17, 2024-07-01
Consolidado EE             2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)   2023-01-17, 2024-07-01
Totalizadores Energia      2023-01-01  2024-10-26  4 días

In [8]:
dias_sin_23_59_lista

[datetime.date(2023, 2, 28),
 datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 19),
 datetime.date(2024, 10, 26)]

In [9]:
dias_con_horas_faltantes_con_23_59

[datetime.date(2023, 1, 17), datetime.date(2024, 7, 1)]

#### Código para detectar días daltantes dentro del primer y último dato resgistrado

In [10]:
if 'dfs_2023_2024' not in globals() or not isinstance(dfs_2023_2024, dict) or not dfs_2023_2024:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2023_2024.keys()):
        df = dfs_2023_2024[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2023_2024.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2023-01-01  2024-04-26                                                 Ninguno
Consolidado Agua           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Aire           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado EE             2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado GasVapor       2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado KPI            2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Produccion     2023-01-01  2024-10-26  188 día

In [11]:
dias_faltantes_lista

[datetime.date(2023, 3, 31),
 datetime.date(2023, 5, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2023, 12, 31),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 1, 2),
 datetime.date(2024, 1, 3),
 datetime.date(2024, 1, 4),
 datetime.date(2024, 1, 5),
 datetime.date(2024, 1, 6),
 datetime.date(2024, 1, 7),
 datetime.date(2024, 1, 8),
 datetime.date(2024, 1, 9),
 datetime.date(2024, 1, 10),
 datetime.date(2024, 1, 11),
 datetime.date(2024, 1, 12),
 datetime.date(2024, 1, 13),
 datetime.date(2024, 1, 14),
 datetime.date(2024, 1, 15),
 datetime.date(2024, 1, 16),
 datetime.date(2024, 1, 17),
 datetime.date(2024, 1, 18),
 datetime.date(2024, 1, 19),
 datetime.date(2024, 1, 20),
 datetime.date(2024, 1, 21),
 datetime.date(2024, 1, 22),
 datetime.date(2024, 1, 23),
 datetime.date(2024, 1, 24),
 datetime.date(2024, 1, 25),
 datetime.date(2024, 1, 26),
 datetime.date(2024, 1, 27),
 datetime.date(2024, 1, 28),
 datetime.date(2024, 1, 29),
 datetime.date(2024, 1, 30),
 datetime.date(2024, 

#### Filtro los días que tienen menos de 5 días consecutivos faltantes. Para solo tener los días que estan fuera del rango de no trabajo de la empresa

In [12]:
import datetime

# Si son 5 o más, se elimina.
LIMITE_CONSECUTIVOS = 5

dias_faltantes_filtrados = []
grupo_actual = []
one_day = datetime.timedelta(days=1)

# Asegurarse de que la lista no esté vacía
if dias_faltantes_lista:
    
    for dia in dias_faltantes_lista:
        # Si el grupo está vacío O el día actual es consecutivo al último del grupo
        if not grupo_actual or (dia - grupo_actual[-1]) == one_day:
            grupo_actual.append(dia)
        else:
            # El día NO es consecutivo. Cerramos el grupo anterior.
            
            # 1. Verificamos el tamaño del grupo que acabamos de cerrar
            if len(grupo_actual) < LIMITE_CONSECUTIVOS:
                dias_faltantes_filtrados.extend(grupo_actual) # Era pequeño, lo guardamos
            
            # 2. Empezamos un nuevo grupo con el día actual
            grupo_actual = [dia]

    # Al final del bucle, debemos verificar el último grupo que quedó abierto
    if len(grupo_actual) < LIMITE_CONSECUTIVOS:
        dias_faltantes_filtrados.extend(grupo_actual)

print("Días faltantes (filtrados, solo grupos < 5 días):")
dias_faltantes_filtrados

Días faltantes (filtrados, solo grupos < 5 días):


[datetime.date(2023, 3, 31),
 datetime.date(2023, 5, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2024, 8, 30),
 datetime.date(2024, 8, 31)]

#### Código para detectar la mala monotonía de los datos

In [13]:
import pandas as pd
import numpy as np # Necesario para np.nan

# Parámetros
# Usar los nombres de columnas tal como aparecen en los DataFrames crudos
DAY_COL  = "DIA"
HOUR_COL = "HORA"
VAL_COL  = "Frio (Kw)"
EPS = 1e-6  # tolerancia numérica

def _to_minutes(h):
    # admite "HH:MM" o "HH:MM:SS"
    try:
        parts = str(h).split(":")
        hh, mm = int(parts[0]), int(parts[1])
        ss = int(parts[2]) if len(parts) > 2 else 0
        return hh*60 + mm + ss/60
    except Exception:
        return np.nan

# --- Simulación (solo por si 'dfs_2023_2024' no existe) ---
try:
    df_original = dfs_2023_2024["Consolidado EE"].copy()
except NameError:
    print("ADVERTENCIA: Usando datos de simulación.")
    df_original = pd.DataFrame({
        "DIA": ["2023-01-01", "2023-01-01", "2023-01-02", "2023-01-02", "2023-01-02"],
        "HORA": ["08:00", "09:00", "08:00", "09:00", "10:00"],
        "Frio (Kw)": [100.0, 101.0, 150.0, 149.0, 151.0] # Día 2 tiene mala monotonía
    })
# --- Fin Simulación ---


# 1) Tomamos el df
df = df_original.copy()

# 2) Normalizamos fecha/hora y ordenamos
df["_dia"]  = pd.to_datetime(df[DAY_COL], errors="coerce", dayfirst=True).dt.date
df["_mins"] = df[HOUR_COL].map(_to_minutes)
df = df.dropna(subset=["_dia","_mins"]).sort_values(["_dia","_mins"])


# 3) Chequeos por día (MODIFICADO)
dias_mala_monotonia_lista = []

for dia, g in df.groupby("_dia", sort=True):
    # Convertimos la columna a números
    s = pd.to_numeric(g.get(VAL_COL, pd.Series([], dtype="float")), errors="coerce").fillna(0.0).values
    
    if len(s) == 0:
        continue

    # Chequeamos solo la monotonía
    # .diff() calcula la resta con el valor anterior. Si es negativo, el valor bajó.
    hubo_caida = (pd.Series(s).diff().fillna(0) < -EPS).any()
    
    if hubo_caida:
        dias_mala_monotonia_lista.append(dia)

In [14]:
dias_mala_monotonia_lista

[datetime.date(2023, 4, 30),
 datetime.date(2023, 5, 20),
 datetime.date(2023, 6, 30),
 datetime.date(2023, 7, 3),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 11, 30),
 datetime.date(2024, 9, 30)]

#### Códgio para hacer un df filtrado con los datos en la hora 23:59 

dias_mala_monotonia_lista -> Toma el primer valor

dias_sin_23_59_lista , dias_con_horas_faltantes_con_23_59  y  dias_faltantes_filtrados  --> Los coloca en NaN

In [15]:
import pandas as pd
import numpy as np
import datetime # Necesario

# 1. Definir columnas y funciones helper
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def _to_date(x):
    try:
        return pd.to_datetime(x, errors="coerce").date()
    except Exception:
        return pd.NaT

def _to_minutes(h):
    try:
        parts = str(h).split(":")
        hh, mm = int(parts[0]), int(parts[1])
        ss = int(parts[2]) if len(parts) > 2 else 0
        return hh*60 + mm + ss/60
    except Exception:
        return np.nan

# --- Simulación (Solo para probar. Borra esto si tenés las listas cargadas) ---
try:
    dias_faltantes_filtrados
except NameError:
    print("ADVERTENCIA: Usando datos de simulación.")
    dias_faltantes_filtrados = [datetime.date(2023, 3, 31)]
    dias_con_horas_faltantes_con_23_59 = [datetime.date(2023, 1, 17)]
    dias_sin_23_59_lista = [datetime.date(2023, 2, 28), datetime.date(2023, 4, 13)]
    dias_mala_monotonia_lista = [datetime.date(2023, 4, 30)]
    df_simulado = pd.DataFrame({
        "DIA": ["2023-01-16", "2023-01-17", "2023-02-28", "2023-04-30"],
        "HORA": ["23:59:00", "23:59:00", "20:00:00", "23:59:00"],
        "Planta (Kw)": [100, 110, 90, 50]
    })
    dfs_2023_2024 = {"Consolidado EE": df_simulado}
# --- Fin Simulación ---


# 2. Unificar las listas de "días malos"
lista_para_nan = set(dias_faltantes_filtrados) | \
                 set(dias_con_horas_faltantes_con_23_59) | \
                 set(dias_sin_23_59_lista)

lista_mala_monotonia = set(dias_mala_monotonia_lista)

print(f"\nSe anularán los datos de {len(lista_para_nan)} días únicos.")
print(f"Se usará 'keep=first' para {len(lista_mala_monotonia)} días con mala monotonía.")

# 3. Procesar el diccionario de DataFrames
dfs_23_59_filtrado = {} 
hojas_saltadas = []

for hoja, df in dfs_2023_2024.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    # --- Pre-procesamiento ---
    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)
    tmp["_ord"] = np.arange(len(tmp)) 
    tmp = tmp.dropna(subset=["_dia", "_mins"])
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # --- A. Filtro principal: Quedarse solo con 23:59 ---
    minuto_23_59 = 23*60 + 59
    df_23_59 = tmp[(tmp["_mins"] >= minuto_23_59) & (tmp["_mins"] < minuto_23_59 + 1)].copy()

    # --- B. Manejo de duplicados (Tu lógica de 'first' y 'last') ---
    mask_mono = df_23_59["_dia"].isin(lista_mala_monotonia)
    df_malos_mono = df_23_59[mask_mono]
    df_buenos = df_23_59[~mask_mono]
    
    df_malos_filtrado = df_malos_mono.drop_duplicates(subset=["_dia"], keep='first')
    df_buenos_filtrado = df_buenos.drop_duplicates(subset=["_dia"], keep='last')
    
    df_unificado_dia = pd.concat([df_buenos_filtrado, df_malos_filtrado])
    
    # --- B-bis. ¡LA MODIFICACIÓN QUE PEDISTE! ---
    # Crear las filas para los días de la lista_para_nan que SÍ faltan
    
    dias_ya_en_df = set(df_unificado_dia["_dia"])
    dias_a_crear = lista_para_nan - dias_ya_en_df
    
    if dias_a_crear:
        print(f"[{hoja}] Creando {len(dias_a_crear)} filas 'NaN' para días que no tenían 23:59...")
        df_dias_faltantes = pd.DataFrame(dias_a_crear, columns=["_dia"])
        
        # Unimos las filas 23:59 existentes con las nuevas filas vacías
        df_unificado_dia = pd.concat([df_unificado_dia, df_dias_faltantes], ignore_index=True)

        # Rellenamos DIA y HORA para esas filas nuevas
        df_unificado_dia[DAY_COL] = df_unificado_dia[DAY_COL].fillna(df_unificado_dia["_dia"])
        df_unificado_dia[HOUR_COL] = df_unificado_dia[HOUR_COL].fillna("23:59:00")
    
    # --- C. Anulación de datos (Preparar para interpolar) ---
    # (Este paso sigue siendo necesario para los días que SÍ tenían 23:59
    #  pero estaban en 'lista_para_nan', como 'dias_con_horas_faltantes...')
    
    cols_para_nan = [c for c in df.columns if c not in [DAY_COL, HOUR_COL]]
    mask_nan = df_unificado_dia["_dia"].isin(lista_para_nan)
    
    df_unificado_dia.loc[mask_nan, cols_para_nan] = np.nan

    # --- D. Limpieza final ---
    columnas_finales = df.columns.tolist()
    df_final = df_unificado_dia[columnas_finales].sort_values(DAY_COL).reset_index(drop=True)

    dfs_23_59_filtrado[hoja] = df_final

print("\n--- Proceso de filtrado a 23:59 completado (CON CREACIÓN DE FILAS) ---")
print(f"Hojas procesadas y guardadas en 'dfs_23_59_filtrado': {len(dfs_23_59_filtrado)}")
print(f"Hojas saltadas: {len(hojas_saltadas)}")

# Verificación de simulación
if "Consolidado EE" in dfs_23_59_filtrado:
    print("\n--- Ejemplo de resultado (Consolidado EE) ---")
    print(dfs_23_59_filtrado["Consolidado EE"].to_string())


Se anularán los datos de 11 días únicos.
Se usará 'keep=first' para 7 días con mala monotonía.
[Consolidado KPI] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Produccion] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Produccion] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado EE] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Energia] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Agua] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Agua] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado GasVapor] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Gas y Vapor] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Aire] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Aire] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores CO2] Creando 8 filas 'NaN' para días que n

#### Código para interpolar los datos faltantes

In [16]:
import pandas as pd
import numpy as np

DAY_COL  = "DIA"
HOUR_COL = "HORA"

def interpolar_nans_existentes(df, day_col=DAY_COL, hour_col=HOUR_COL):
    g = df.copy()

    # Convertimos la columna 'DIA' en un datetime real.
    g['_fecha'] = pd.to_datetime(g[day_col], errors='coerce', dayfirst=True)
    
    # Nos aseguramos de que no haya fechas nulas y establecemos esa fecha como el ÍNDICE. Esto es OBLIGATORIO para que method="time" funcione.
    g = g.dropna(subset=['_fecha']).set_index('_fecha')

    # --- 2. Interpolar SOLO columnas numéricas ---
    num_cols = g.select_dtypes(include="number").columns
    
    if len(num_cols) > 0:
        # .interpolate() solo rellena los NaN. Los días con datos (los "buenos") se usan como referencia.
        # method="time" calcula la distancia entre las fechas.
        g[num_cols] = g[num_cols].interpolate(method="time")
    
        # Si el primer o último día era NaN, la interpolación no puede rellenarlos. Usamos ffill (forward-fill) 
        # y bfill (backward-fill) para rellenar esos huecos.
        g[num_cols] = g[num_cols].ffill().bfill()

    # Nos aseguramos de que todas las filas tengan la hora 23:59
    if hour_col in g.columns:
        g[hour_col] = g[hour_col].fillna("23:59:00")

    # Devolvemos el DataFrame a su estado original (sin índice _fecha)
    g = g.reset_index(drop=True) 
    return g

print("Iniciando interpolación de datos...")
dfs_interpolado = {}

# Iteramos sobre el diccionario que ya filtramos a 23:59
for nombre, df in dfs_23_59_filtrado.items():
    if df.empty:
        continue
    print(f"Interpolando hoja: {nombre}...")
    dfs_interpolado[nombre] = interpolar_nans_existentes(df)

print("\n--- Interpolación completada ---")
print(f"Resultados guardados en el diccionario 'dfs_interpolado'.")

# Para verificar, podés imprimir el resultado de la simulación:
if "Consolidado EE" in dfs_interpolado:
    print("\n--- Ejemplo de resultado interpolado (Consolidado EE) ---")
    print(dfs_interpolado["Consolidado EE"].to_string())

#sobre escribir el diccionario original
dfs_2023_2024 = dfs_interpolado

Iniciando interpolación de datos...
Interpolando hoja: Consolidado KPI...
Interpolando hoja: Consolidado Produccion...
Interpolando hoja: Totalizadores Produccion...
Interpolando hoja: Consolidado EE...
Interpolando hoja: Totalizadores Energia...
Interpolando hoja: Consolidado Agua...
Interpolando hoja: Totalizadores Agua...
Interpolando hoja: Consolidado GasVapor...
Interpolando hoja: Totalizadores Gas y Vapor...
Interpolando hoja: Consolidado Aire...
Interpolando hoja: Totalizadores Aire...
Interpolando hoja: Totalizadores CO2...
Interpolando hoja: Totalizadores Efluentes...
Interpolando hoja: Totalizadores Glicol...
Interpolando hoja: Seguimiento Dia...
Interpolando hoja: Auxiliar...

--- Interpolación completada ---
Resultados guardados en el diccionario 'dfs_interpolado'.

--- Ejemplo de resultado interpolado (Consolidado EE) ---
           DIA      HORA   Planta (Kw)  Elaboracion (Kw)  Bodega (Kw)  Cocina (Kw)  Envasado (Kw)  Linea 2 (Kw)  Linea 3 (Kw)  Linea 4 (Kw)  Servicios (K

Código para verificar fechas

In [17]:
fecha_buscar = "2023-05-31"

print(f"\n--- Datos para el día {fecha_buscar} ---")
for nombre_hoja, df in dfs_2023_2024.items():
    # Verificar si el DataFrame tiene la columna DIA
    if "DIA" in df.columns:
        # Filtrar por la fecha específica
        datos_dia = df[df["DIA"].dt.strftime("%Y-%m-%d") == fecha_buscar]
        
        if not datos_dia.empty:
            print(f"\nHoja: {nombre_hoja}")
            print(f"Registros encontrados: {len(datos_dia)}")
            print(datos_dia.to_string())
    else:
        print(f"\nLa hoja {nombre_hoja} no tiene columna DIA")


--- Datos para el día 2023-05-31 ---

Hoja: Consolidado KPI
Registros encontrados: 1
           DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L

### Datos 2022-2023

In [18]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data\Totalizadores Planta de Cerveza - 2022_2023.xlsx')
# 2) Crear un dict con un DataFrame por hoja
dfs_2022_2023 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2022_2023[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_4528\283502712.py:2: SyntaxWarning: invalid escape sequence '\T'
  xls = pd.ExcelFile('data\Totalizadores Planta de Cerveza - 2022_2023.xlsx')


--- Resumen de hojas y columnas ---
                         hoja  filas  columnas                                                                                                                                                                                         nombres_columnas
0             Consolidado KPI  15317       123  DIA, HORA, EE Planta / Hl, EE Elaboracion / Hl, EE Bodega / Hl, EE Cocina / Hl, EE Envasado / Hl, EE Linea 2 / Hl, EE Linea 3 / Hl, EE Linea 4 / Hl, EE Linea 5 / Hl, EE Servicios / Hl, EE Sala Maq...
1                       Metas     36        57  Mes / Año, Año + Mes, Agua Planta, EE Planta, ET Planta, Aire Planta, Unnamed: 6, Meta Agua Elab, Meta Agua Bodega, Meta Agua Cocina, Meta Agua Envas, Meta Agua Linea 2, Meta Agua Linea 3, Meta Ag...
2      Consolidado Produccion  15450        14  DIA, HORA, Hl de Mosto, Hl Cerveza Cocina, Hl Producido Bodega, Hl Cerveza Filtrada, Hl Cerveza Envasada, Hl Cerveza L2, Hl Cerveza L3, Hl Cerveza L4, Hl Cerveza L5, Cocimi

#### Código para ver una hoja específica

In [19]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2022_2023:

    df_hoja = dfs_2022_2023[nombre_hoja_para_ver]
    
    print(f"Mostrando la hoja: {nombre_hoja_para_ver}")
    print(f"Dimensiones: {df_hoja.shape[0]} filas x {df_hoja.shape[1]} columnas")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(df_hoja.head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(df_hoja.tail(3).to_string())
else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2022_2023.keys()))

Mostrando la hoja: Consolidado KPI
Dimensiones: 15317 filas x 123 columnas

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / H

#### Código para detectar días con horas faltantes

In [20]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2022_2023.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_set = set(todos_los_dias) - set(dias_con_23_59)
        dias_sin_23_59_lista = sorted(list(dias_sin_23_59_set))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_con_23_59 = sorted(list( set(dias_con_horas_faltantes_sr.index) - dias_sin_23_59_set ))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_con_23_59)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59            dias_con_horas_faltantes
hoja                                                                                                                                       
Consolidado KPI            2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  2022-07-01, 2022-11-01, 2023-01-17
Metas                      2021-01-01  2023-12-01                                 N/A (Hoja no horaria)               N/A (Hoja no horaria)
Consolidado Produccion     2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  2022-07-01, 2022-11-01, 2023-01-17
Totalizadores Produccion   2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  2022-07-01, 2022-11-01, 2023-01-17
Consolidado EE             2022-01-01  2023-12-30  4 días (Ej: 2022-03-02, 2022-07-13, 2023-02-28, ...)  2022-07-01, 202

In [21]:
dias_sin_23_59_lista

[datetime.date(2022, 3, 2),
 datetime.date(2022, 7, 13),
 datetime.date(2023, 2, 28),
 datetime.date(2023, 3, 6)]

In [22]:
dias_con_horas_faltantes_con_23_59

[datetime.date(2022, 7, 1),
 datetime.date(2022, 11, 1),
 datetime.date(2023, 1, 17)]

#### Código para detectar días daltantes dentro del primer y último dato resgistrado

In [23]:
if 'dfs_2022_2023' not in globals() or not isinstance(dfs_2022_2023, dict) or not dfs_2022_2023:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2022_2023.keys()):
        df = dfs_2022_2023[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2022_2023.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Agua           2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Aire           2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado EE             2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado GasVapor       2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado KPI            2022-01-01  2023-12-30  121 días (Ej: 2022-03-31, 2022-05-31, 2022-10-31, ...)
Consolidado Produccion     2022-01-01  2023-12-30  121 día

In [24]:
dias_faltantes_lista

[datetime.date(2022, 3, 31),
 datetime.date(2022, 5, 31),
 datetime.date(2022, 10, 31),
 datetime.date(2022, 12, 31),
 datetime.date(2023, 3, 7),
 datetime.date(2023, 3, 8),
 datetime.date(2023, 3, 9),
 datetime.date(2023, 3, 10),
 datetime.date(2023, 3, 11),
 datetime.date(2023, 3, 12),
 datetime.date(2023, 3, 13),
 datetime.date(2023, 3, 14),
 datetime.date(2023, 3, 15),
 datetime.date(2023, 3, 16),
 datetime.date(2023, 3, 17),
 datetime.date(2023, 3, 18),
 datetime.date(2023, 3, 19),
 datetime.date(2023, 3, 20),
 datetime.date(2023, 3, 21),
 datetime.date(2023, 3, 22),
 datetime.date(2023, 3, 23),
 datetime.date(2023, 3, 24),
 datetime.date(2023, 3, 25),
 datetime.date(2023, 3, 26),
 datetime.date(2023, 3, 27),
 datetime.date(2023, 3, 28),
 datetime.date(2023, 3, 29),
 datetime.date(2023, 3, 30),
 datetime.date(2023, 3, 31),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 4, 2),
 datetime.date(2023, 4, 3),
 datetime.date(2023, 4, 4),
 datetime.date(2023, 4, 5),
 datetime.date(2023,

#### Filtro los días que tienen menos de 5 días consecutivos faltantes. Para solo tener los días que estan fuera del rango de no trabajo de la empresa

In [25]:
import datetime

# Si son 5 o más, se elimina.
LIMITE_CONSECUTIVOS = 5

dias_faltantes_filtrados = []
grupo_actual = []
one_day = datetime.timedelta(days=1)

# Asegurarse de que la lista no esté vacía
if dias_faltantes_lista:
    
    for dia in dias_faltantes_lista:
        # Si el grupo está vacío O el día actual es consecutivo al último del grupo
        if not grupo_actual or (dia - grupo_actual[-1]) == one_day:
            grupo_actual.append(dia)
        else:
            # El día NO es consecutivo. Cerramos el grupo anterior.
            
            # 1. Verificamos el tamaño del grupo que acabamos de cerrar
            if len(grupo_actual) < LIMITE_CONSECUTIVOS:
                dias_faltantes_filtrados.extend(grupo_actual) # Era pequeño, lo guardamos
            
            # 2. Empezamos un nuevo grupo con el día actual
            grupo_actual = [dia]

    # Al final del bucle, debemos verificar el último grupo que quedó abierto
    if len(grupo_actual) < LIMITE_CONSECUTIVOS:
        dias_faltantes_filtrados.extend(grupo_actual)

print("Días faltantes (filtrados, solo grupos < 5 días):")
dias_faltantes_filtrados

Días faltantes (filtrados, solo grupos < 5 días):


[datetime.date(2022, 3, 31),
 datetime.date(2022, 5, 31),
 datetime.date(2022, 10, 31),
 datetime.date(2022, 12, 31),
 datetime.date(2023, 10, 31)]

#### Código para detectar la mala monotonía de los datos

In [26]:
import pandas as pd
import numpy as np # Necesario para np.nan

# Parámetros
# Usar los nombres de columnas tal como aparecen en los DataFrames crudos
DAY_COL  = "DIA"
HOUR_COL = "HORA"
VAL_COL  = "Frio (Kw)"
EPS = 1e-6  # tolerancia numérica

def _to_minutes(h):
    # admite "HH:MM" o "HH:MM:SS"
    try:
        parts = str(h).split(":")
        hh, mm = int(parts[0]), int(parts[1])
        ss = int(parts[2]) if len(parts) > 2 else 0
        return hh*60 + mm + ss/60
    except Exception:
        return np.nan

# 1) Tomamos el df
df = df_original.copy()

# 2) Normalizamos fecha/hora y ordenamos
df["_dia"]  = pd.to_datetime(df[DAY_COL], errors="coerce", dayfirst=True).dt.date
df["_mins"] = df[HOUR_COL].map(_to_minutes)
df = df.dropna(subset=["_dia","_mins"]).sort_values(["_dia","_mins"])


# 3) Chequeos por día (MODIFICADO)
dias_mala_monotonia_lista = []

for dia, g in df.groupby("_dia", sort=True):
    # Convertimos la columna a números
    s = pd.to_numeric(g.get(VAL_COL, pd.Series([], dtype="float")), errors="coerce").fillna(0.0).values
    
    if len(s) == 0:
        continue

    # Chequeamos solo la monotonía
    # .diff() calcula la resta con el valor anterior. Si es negativo, el valor bajó.
    hubo_caida = (pd.Series(s).diff().fillna(0) < -EPS).any()
    
    if hubo_caida:
        dias_mala_monotonia_lista.append(dia)

In [27]:
dias_mala_monotonia_lista

[datetime.date(2023, 4, 30),
 datetime.date(2023, 5, 20),
 datetime.date(2023, 6, 30),
 datetime.date(2023, 7, 3),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 11, 30),
 datetime.date(2024, 9, 30)]

#### Códgio para hacer un df filtrado con los datos en la hora 23:59 

dias_mala_monotonia_lista -> Toma el primer valor

dias_sin_23_59_lista , dias_con_horas_faltantes_con_23_59  y  dias_faltantes_filtrados  --> Los coloca en NaN

In [28]:
import pandas as pd
import numpy as np
import datetime # Necesario

# 1. Definir columnas y funciones helper
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def _to_date(x):
    try:
        return pd.to_datetime(x, errors="coerce").date()
    except Exception:
        return pd.NaT

# --- Simulación (Solo para probar. Borra esto si tenés las listas cargadas) ---
try:
    dias_faltantes_filtrados
except NameError:
    print("ADVERTENCIA: Usando datos de simulación.")
    dias_faltantes_filtrados = [datetime.date(2023, 3, 31)]
    dias_con_horas_faltantes_con_23_59 = [datetime.date(2023, 1, 17)]
    dias_sin_23_59_lista = [datetime.date(2023, 2, 28), datetime.date(2023, 4, 13)]
    dias_mala_monotonia_lista = [datetime.date(2023, 4, 30)]
    df_simulado = pd.DataFrame({
        "DIA": ["2023-01-16", "2023-01-17", "2023-02-28", "2023-04-30"],
        "HORA": ["23:59:00", "23:59:00", "20:00:00", "23:59:00"],
        "Planta (Kw)": [100, 110, 90, 50]
    })
    dfs_2022_2023 = {"Consolidado EE": df_simulado}
# --- Fin Simulación ---


# 2. Unificar las listas de "días malos"
lista_para_nan = set(dias_faltantes_filtrados) | \
                 set(dias_con_horas_faltantes_con_23_59) | \
                 set(dias_sin_23_59_lista)

lista_mala_monotonia = set(dias_mala_monotonia_lista)

print(f"\nSe anularán los datos de {len(lista_para_nan)} días únicos.")
print(f"Se usará 'keep=first' para {len(lista_mala_monotonia)} días con mala monotonía.")

# 3. Procesar el diccionario de DataFrames
dfs_23_59_filtrado = {} 
hojas_saltadas = []

for hoja, df in dfs_2022_2023.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    # --- Pre-procesamiento ---
    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)
    tmp["_ord"] = np.arange(len(tmp)) 
    tmp = tmp.dropna(subset=["_dia", "_mins"])
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # --- A. Filtro principal: Quedarse solo con 23:59 ---
    minuto_23_59 = 23*60 + 59
    df_23_59 = tmp[(tmp["_mins"] >= minuto_23_59) & (tmp["_mins"] < minuto_23_59 + 1)].copy()

    # --- B. Manejo de duplicados (Tu lógica de 'first' y 'last') ---
    mask_mono = df_23_59["_dia"].isin(lista_mala_monotonia)
    df_malos_mono = df_23_59[mask_mono]
    df_buenos = df_23_59[~mask_mono]
    
    df_malos_filtrado = df_malos_mono.drop_duplicates(subset=["_dia"], keep='first')
    df_buenos_filtrado = df_buenos.drop_duplicates(subset=["_dia"], keep='last')
    
    df_unificado_dia = pd.concat([df_buenos_filtrado, df_malos_filtrado])
    
    # --- B-bis. ¡LA MODIFICACIÓN QUE PEDISTE! ---
    # Crear las filas para los días de la lista_para_nan que SÍ faltan
    
    dias_ya_en_df = set(df_unificado_dia["_dia"])
    dias_a_crear = lista_para_nan - dias_ya_en_df
    
    if dias_a_crear:
        print(f"[{hoja}] Creando {len(dias_a_crear)} filas 'NaN' para días que no tenían 23:59...")
        df_dias_faltantes = pd.DataFrame(dias_a_crear, columns=["_dia"])
        
        # Unimos las filas 23:59 existentes con las nuevas filas vacías
        df_unificado_dia = pd.concat([df_unificado_dia, df_dias_faltantes], ignore_index=True)

        # Rellenamos DIA y HORA para esas filas nuevas
        df_unificado_dia[DAY_COL] = df_unificado_dia[DAY_COL].fillna(df_unificado_dia["_dia"])
        df_unificado_dia[HOUR_COL] = df_unificado_dia[HOUR_COL].fillna("23:59:00")
    
    # --- C. Anulación de datos (Preparar para interpolar) ---
    # (Este paso sigue siendo necesario para los días que SÍ tenían 23:59
    #  pero estaban en 'lista_para_nan', como 'dias_con_horas_faltantes...')
    
    cols_para_nan = [c for c in df.columns if c not in [DAY_COL, HOUR_COL]]
    mask_nan = df_unificado_dia["_dia"].isin(lista_para_nan)
    
    df_unificado_dia.loc[mask_nan, cols_para_nan] = np.nan

    # --- D. Limpieza final ---
    columnas_finales = df.columns.tolist()
    df_final = df_unificado_dia[columnas_finales].sort_values(DAY_COL).reset_index(drop=True)

    dfs_23_59_filtrado[hoja] = df_final

print("\n--- Proceso de filtrado a 23:59 completado (CON CREACIÓN DE FILAS) ---")
print(f"Hojas procesadas y guardadas en 'dfs_23_59_filtrado': {len(dfs_23_59_filtrado)}")
print(f"Hojas saltadas: {len(hojas_saltadas)}")

# Verificación de simulación
if "Consolidado EE" in dfs_23_59_filtrado:
    print("\n--- Ejemplo de resultado (Consolidado EE) ---")
    print(dfs_23_59_filtrado["Consolidado EE"].to_string())


Se anularán los datos de 12 días únicos.
Se usará 'keep=first' para 7 días con mala monotonía.
[Consolidado KPI] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Produccion] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Produccion] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado EE] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Energia] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Agua] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Agua] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado GasVapor] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Gas y Vapor] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Consolidado Aire] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Aire] Creando 9 filas 'NaN' para días que no tenían 23:59...
[Totalizadores CO2] Creando 8 filas 'NaN' para días que n

#### Código para interpolar los datos faltantes

In [29]:
import pandas as pd
import numpy as np

DAY_COL  = "DIA"
HOUR_COL = "HORA"

def interpolar_nans_existentes(df, day_col=DAY_COL, hour_col=HOUR_COL):
    g = df.copy()

    # Convertimos la columna 'DIA' en un datetime real.
    g['_fecha'] = pd.to_datetime(g[day_col], errors='coerce', dayfirst=True)
    
    # Nos aseguramos de que no haya fechas nulas y establecemos esa fecha como el ÍNDICE. Esto es OBLIGATORIO para que method="time" funcione.
    g = g.dropna(subset=['_fecha']).set_index('_fecha')

    # --- 2. Interpolar SOLO columnas numéricas ---
    num_cols = g.select_dtypes(include="number").columns
    
    if len(num_cols) > 0:
        # .interpolate() solo rellena los NaN. Los días con datos (los "buenos") se usan como referencia.
        # method="time" calcula la distancia entre las fechas.
        g[num_cols] = g[num_cols].interpolate(method="time")
    
        # Si el primer o último día era NaN, la interpolación no puede rellenarlos. Usamos ffill (forward-fill) 
        # y bfill (backward-fill) para rellenar esos huecos.
        g[num_cols] = g[num_cols].ffill().bfill()

    # Nos aseguramos de que todas las filas tengan la hora 23:59
    if hour_col in g.columns:
        g[hour_col] = g[hour_col].fillna("23:59:00")

    # Devolvemos el DataFrame a su estado original (sin índice _fecha)
    g = g.reset_index(drop=True) 
    return g

print("Iniciando interpolación de datos...")
dfs_interpolado = {}

# Iteramos sobre el diccionario que ya filtramos a 23:59
for nombre, df in dfs_23_59_filtrado.items():
    if df.empty:
        continue
    print(f"Interpolando hoja: {nombre}...")
    dfs_interpolado[nombre] = interpolar_nans_existentes(df)

print("\n--- Interpolación completada ---")
print(f"Resultados guardados en el diccionario 'dfs_interpolado'.")

# Para verificar, podés imprimir el resultado de la simulación:
if "Consolidado EE" in dfs_interpolado:
    print("\n--- Ejemplo de resultado interpolado (Consolidado EE) ---")
    print(dfs_interpolado["Consolidado EE"].to_string())

#sobre escribir el diccionario original
dfs_2022_2023 = dfs_interpolado

Iniciando interpolación de datos...
Interpolando hoja: Consolidado KPI...
Interpolando hoja: Consolidado Produccion...
Interpolando hoja: Totalizadores Produccion...
Interpolando hoja: Consolidado EE...
Interpolando hoja: Totalizadores Energia...
Interpolando hoja: Consolidado Agua...
Interpolando hoja: Totalizadores Agua...
Interpolando hoja: Consolidado GasVapor...
Interpolando hoja: Totalizadores Gas y Vapor...
Interpolando hoja: Consolidado Aire...
Interpolando hoja: Totalizadores Aire...
Interpolando hoja: Totalizadores CO2...
Interpolando hoja: Totalizadores Efluentes...
Interpolando hoja: Totalizadores Glicol...
Interpolando hoja: Seguimiento Dia...
Interpolando hoja: Auxiliar...

--- Interpolación completada ---
Resultados guardados en el diccionario 'dfs_interpolado'.

--- Ejemplo de resultado interpolado (Consolidado EE) ---
           DIA      HORA   Planta (Kw)  Elaboracion (Kw)   Bodega (Kw)  Cocina (Kw)  Envasado (Kw)  Linea 2 (Kw)  Linea 3 (Kw)  Linea 4 (Kw)  Servicios (

Código para verificar fechas

In [30]:
fecha_buscar = "2022-05-31"

print(f"\n--- Datos para el día {fecha_buscar} ---")
for nombre_hoja, df in dfs_2022_2023.items():
    # Verificar si el DataFrame tiene la columna DIA
    if "DIA" in df.columns:
        # Filtrar por la fecha específica
        datos_dia = df[df["DIA"].dt.strftime("%Y-%m-%d") == fecha_buscar]
        
        if not datos_dia.empty:
            print(f"\nHoja: {nombre_hoja}")
            print(f"Registros encontrados: {len(datos_dia)}")
            print(datos_dia.to_string())
    else:
        print(f"\nLa hoja {nombre_hoja} no tiene columna DIA")


--- Datos para el día 2022-05-31 ---

Hoja: Consolidado KPI
Registros encontrados: 1
           DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L

### Datos 2021-2022

In [31]:
# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('data\Totalizadores Planta de Cerveza 2021_2022.xlsx')
# 2) Crear un dict con un DataFrame por hoja
dfs_2021_2022 = {}
resumen = []

for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs_2021_2022[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

print("--- Resumen de hojas y columnas ---")
print(resumen_df)

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_4528\676958964.py:2: SyntaxWarning: invalid escape sequence '\T'
  xls = pd.ExcelFile('data\Totalizadores Planta de Cerveza 2021_2022.xlsx')


--- Resumen de hojas y columnas ---
                         hoja  filas  columnas                                                                                                                                                                                         nombres_columnas
0             Consolidado KPI  16049        62  DIA, HORA, EE Planta / Hl, EE Elaboracion / Hl, EE Bodega / Hl, EE Cocina / Hl, EE Envasado / Hl, EE Linea 2 / Hl, EE Linea 3 / Hl, EE Linea 4 / Hl, EE Linea 5 / Hl, EE Servicios / Hl, EE Sala Maq...
1      Consolidado Produccion  15573        12                DIA, HORA, Hl de Mosto, Hl Cerveza Cocina, Hl Producido Bodega, Hl Cerveza Filtrada, Hl Cerveza Envasada, Hl Cerveza L2, Hl Cerveza L3, Hl Cerveza L4, Hl Cerveza L5, Cocimientos Diarios
2    Totalizadores Produccion  15573        40  DIA, HORA, HL Mosto Budweiser, HL Mosto Tecate, HL Mosto Local, HL Mosto Heineken, HL Mosto Negra, HL Mosto Fuerte, HL Mosto Indio, HL Mosto Palermo, HL Mosto Bieckert, HL 

#### Código para ver una hoja específica

In [32]:
nombre_hoja_para_ver = 'Consolidado KPI'

if nombre_hoja_para_ver in dfs_2021_2022:

    df_hoja = dfs_2021_2022[nombre_hoja_para_ver]
    
    print(f"Mostrando la hoja: {nombre_hoja_para_ver}")
    print(f"Dimensiones: {df_hoja.shape[0]} filas x {df_hoja.shape[1]} columnas")
    
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 3 FILAS (.head()) ---")
    print(df_hoja.head(3).to_string())
    
    print("\n\n--- ÚLTIMAS 3 FILAS (.tail()) ---")
    print(df_hoja.tail(3).to_string())
else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs_2021_2022.keys()))

Mostrando la hoja: Consolidado KPI
Dimensiones: 16049 filas x 62 columnas

--- PRIMERAS 3 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl

#### Código para detectar días con horas faltantes

In [33]:
print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs_2021_2022.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_set = set(todos_los_dias) - set(dias_con_23_59)
        dias_sin_23_59_lista = sorted(list(dias_sin_23_59_set))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_con_23_59 = sorted(list( set(dias_con_horas_faltantes_sr.index) - dias_sin_23_59_set ))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_con_23_59)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59 dias_con_horas_faltantes
hoja                                                                                                                            
Consolidado KPI            2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13   2022-07-01, 2022-11-01
Consolidado Produccion     2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13   2022-07-01, 2022-11-01
Totalizadores Produccion   2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13   2022-07-01, 2022-11-01
Consolidado EE             2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13               2022-11-01
Totalizadores Energia      2021-01-01  2022-12-30                    2022-03-02, 2022-03-16, 2022-07-13               2022-11-01
Consolidado Agua           2021-01-01  2022-12-30        

In [34]:
dias_sin_23_59_lista

[datetime.date(2021, 7, 1),
 datetime.date(2022, 3, 2),
 datetime.date(2022, 3, 16),
 datetime.date(2022, 7, 13)]

In [35]:
dias_con_horas_faltantes_con_23_59

[datetime.date(2022, 11, 1)]

#### Código para detectar días daltantes dentro del primer y último dato resgistrado

In [36]:
if 'dfs_2021_2022' not in globals() or not isinstance(dfs_2021_2022, dict) or not dfs_2021_2022:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs_2021_2022.keys()):
        df = dfs_2021_2022[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs_2021_2022.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Agua           2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Aire           2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado EE             2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado GasVapor       2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado KPI            2021-01-01  2022-12-30  111 días (Ej: 2021-03-31, 2021-05-31, 2021-10-31, ...)
Consolidado Produccion     2021-01-01  2022-12-30  111 día

In [37]:
dias_faltantes_lista

[datetime.date(2021, 3, 31),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 10, 31),
 datetime.date(2021, 12, 31),
 datetime.date(2022, 3, 17),
 datetime.date(2022, 3, 18),
 datetime.date(2022, 3, 19),
 datetime.date(2022, 3, 20),
 datetime.date(2022, 3, 21),
 datetime.date(2022, 3, 22),
 datetime.date(2022, 3, 23),
 datetime.date(2022, 3, 24),
 datetime.date(2022, 3, 25),
 datetime.date(2022, 3, 26),
 datetime.date(2022, 3, 27),
 datetime.date(2022, 3, 28),
 datetime.date(2022, 3, 29),
 datetime.date(2022, 3, 30),
 datetime.date(2022, 3, 31),
 datetime.date(2022, 4, 1),
 datetime.date(2022, 4, 2),
 datetime.date(2022, 4, 3),
 datetime.date(2022, 4, 4),
 datetime.date(2022, 4, 5),
 datetime.date(2022, 4, 6),
 datetime.date(2022, 4, 7),
 datetime.date(2022, 4, 8),
 datetime.date(2022, 4, 9),
 datetime.date(2022, 4, 10),
 datetime.date(2022, 4, 11),
 datetime.date(2022, 4, 12),
 datetime.date(2022, 4, 13),
 datetime.date(2022, 4, 14),
 datetime.date(2022, 4, 15),
 datetime.date(2022, 

#### Filtro los días que tienen menos de 5 días consecutivos faltantes. Para solo tener los días que estan fuera del rango de no trabajo de la empresa

In [38]:
import datetime

# Si son 5 o más, se elimina.
LIMITE_CONSECUTIVOS = 5

dias_faltantes_filtrados = []
grupo_actual = []
one_day = datetime.timedelta(days=1)

# Asegurarse de que la lista no esté vacía
if dias_faltantes_lista:
    
    for dia in dias_faltantes_lista:
        # Si el grupo está vacío O el día actual es consecutivo al último del grupo
        if not grupo_actual or (dia - grupo_actual[-1]) == one_day:
            grupo_actual.append(dia)
        else:
            # El día NO es consecutivo. Cerramos el grupo anterior.
            
            # 1. Verificamos el tamaño del grupo que acabamos de cerrar
            if len(grupo_actual) < LIMITE_CONSECUTIVOS:
                dias_faltantes_filtrados.extend(grupo_actual) # Era pequeño, lo guardamos
            
            # 2. Empezamos un nuevo grupo con el día actual
            grupo_actual = [dia]

    # Al final del bucle, debemos verificar el último grupo que quedó abierto
    if len(grupo_actual) < LIMITE_CONSECUTIVOS:
        dias_faltantes_filtrados.extend(grupo_actual)

print("Días faltantes (filtrados, solo grupos < 5 días):")
dias_faltantes_filtrados

Días faltantes (filtrados, solo grupos < 5 días):


[datetime.date(2021, 3, 31),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 10, 31),
 datetime.date(2021, 12, 31),
 datetime.date(2022, 10, 31)]

#### Código para detectar la mala monotonía de los datos

In [39]:
import pandas as pd
import numpy as np # Necesario para np.nan

# Parámetros
# Usar los nombres de columnas tal como aparecen en los DataFrames crudos
DAY_COL  = "DIA"
HOUR_COL = "HORA"
VAL_COL  = "Frio (Kw)"
EPS = 1e-6  # tolerancia numérica

def _to_minutes(h):
    # admite "HH:MM" o "HH:MM:SS"
    try:
        parts = str(h).split(":")
        hh, mm = int(parts[0]), int(parts[1])
        ss = int(parts[2]) if len(parts) > 2 else 0
        return hh*60 + mm + ss/60
    except Exception:
        return np.nan

# 1) Tomamos el df
df = df_original.copy()

# 2) Normalizamos fecha/hora y ordenamos
df["_dia"]  = pd.to_datetime(df[DAY_COL], errors="coerce", dayfirst=True).dt.date
df["_mins"] = df[HOUR_COL].map(_to_minutes)
df = df.dropna(subset=["_dia","_mins"]).sort_values(["_dia","_mins"])


# 3) Chequeos por día (MODIFICADO)
dias_mala_monotonia_lista = []

for dia, g in df.groupby("_dia", sort=True):
    # Convertimos la columna a números
    s = pd.to_numeric(g.get(VAL_COL, pd.Series([], dtype="float")), errors="coerce").fillna(0.0).values
    
    if len(s) == 0:
        continue

    # Chequeamos solo la monotonía
    # .diff() calcula la resta con el valor anterior. Si es negativo, el valor bajó.
    hubo_caida = (pd.Series(s).diff().fillna(0) < -EPS).any()
    
    if hubo_caida:
        dias_mala_monotonia_lista.append(dia)

In [40]:
dias_mala_monotonia_lista

[datetime.date(2023, 4, 30),
 datetime.date(2023, 5, 20),
 datetime.date(2023, 6, 30),
 datetime.date(2023, 7, 3),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 11, 30),
 datetime.date(2024, 9, 30)]

#### Códgio para hacer un df filtrado con los datos en la hora 23:59 

dias_mala_monotonia_lista -> Toma el primer valor

dias_sin_23_59_lista , dias_con_horas_faltantes_con_23_59  y  dias_faltantes_filtrados  --> Los coloca en NaN

In [41]:
import pandas as pd
import numpy as np
import datetime # Necesario

# 1. Definir columnas y funciones helper
DAY_COL  = "DIA"
HOUR_COL = "HORA"

def _to_date(x):
    try:
        return pd.to_datetime(x, errors="coerce").date()
    except Exception:
        return pd.NaT

# --- Simulación (Solo para probar. Borra esto si tenés las listas cargadas) ---
try:
    dias_faltantes_filtrados
except NameError:
    print("ADVERTENCIA: Usando datos de simulación.")
    dias_faltantes_filtrados = [datetime.date(2023, 3, 31)]
    dias_con_horas_faltantes_con_23_59 = [datetime.date(2023, 1, 17)]
    dias_sin_23_59_lista = [datetime.date(2023, 2, 28), datetime.date(2023, 4, 13)]
    dias_mala_monotonia_lista = [datetime.date(2023, 4, 30)]
    df_simulado = pd.DataFrame({
        "DIA": ["2023-01-16", "2023-01-17", "2023-02-28", "2023-04-30"],
        "HORA": ["23:59:00", "23:59:00", "20:00:00", "23:59:00"],
        "Planta (Kw)": [100, 110, 90, 50]
    })
    dfs_2021_2022 = {"Consolidado EE": df_simulado}
# --- Fin Simulación ---


# 2. Unificar las listas de "días malos"
lista_para_nan = set(dias_faltantes_filtrados) | \
                 set(dias_con_horas_faltantes_con_23_59) | \
                 set(dias_sin_23_59_lista)

lista_mala_monotonia = set(dias_mala_monotonia_lista)

print(f"\nSe anularán los datos de {len(lista_para_nan)} días únicos.")
print(f"Se usará 'keep=first' para {len(lista_mala_monotonia)} días con mala monotonía.")

# 3. Procesar el diccionario de DataFrames
dfs_23_59_filtrado = {} 
hojas_saltadas = []

for hoja, df in dfs_2021_2022.items():
    if DAY_COL not in df.columns or HOUR_COL not in df.columns:
        hojas_saltadas.append((hoja, "Falta DIA u HORA"))
        continue

    # --- Pre-procesamiento ---
    tmp = df.copy()
    tmp["_dia"]  = tmp[DAY_COL].map(_to_date)
    tmp["_mins"] = tmp[HOUR_COL].map(_to_minutes)
    tmp["_ord"] = np.arange(len(tmp)) 
    tmp = tmp.dropna(subset=["_dia", "_mins"])
    tmp = tmp.sort_values(["_dia", "_mins", "_ord"], kind="stable")

    # --- A. Filtro principal: Quedarse solo con 23:59 ---
    minuto_23_59 = 23*60 + 59
    df_23_59 = tmp[(tmp["_mins"] >= minuto_23_59) & (tmp["_mins"] < minuto_23_59 + 1)].copy()

    # --- B. Manejo de duplicados (Tu lógica de 'first' y 'last') ---
    mask_mono = df_23_59["_dia"].isin(lista_mala_monotonia)
    df_malos_mono = df_23_59[mask_mono]
    df_buenos = df_23_59[~mask_mono]
    
    df_malos_filtrado = df_malos_mono.drop_duplicates(subset=["_dia"], keep='first')
    df_buenos_filtrado = df_buenos.drop_duplicates(subset=["_dia"], keep='last')
    
    df_unificado_dia = pd.concat([df_buenos_filtrado, df_malos_filtrado])
    
    # --- B-bis. ¡LA MODIFICACIÓN QUE PEDISTE! ---
    # Crear las filas para los días de la lista_para_nan que SÍ faltan
    
    dias_ya_en_df = set(df_unificado_dia["_dia"])
    dias_a_crear = lista_para_nan - dias_ya_en_df
    
    if dias_a_crear:
        print(f"[{hoja}] Creando {len(dias_a_crear)} filas 'NaN' para días que no tenían 23:59...")
        df_dias_faltantes = pd.DataFrame(dias_a_crear, columns=["_dia"])
        
        # Unimos las filas 23:59 existentes con las nuevas filas vacías
        df_unificado_dia = pd.concat([df_unificado_dia, df_dias_faltantes], ignore_index=True)

        # Rellenamos DIA y HORA para esas filas nuevas
        df_unificado_dia[DAY_COL] = df_unificado_dia[DAY_COL].fillna(df_unificado_dia["_dia"])
        df_unificado_dia[HOUR_COL] = df_unificado_dia[HOUR_COL].fillna("23:59:00")
    
    # --- C. Anulación de datos (Preparar para interpolar) ---
    # (Este paso sigue siendo necesario para los días que SÍ tenían 23:59
    #  pero estaban en 'lista_para_nan', como 'dias_con_horas_faltantes...')
    
    cols_para_nan = [c for c in df.columns if c not in [DAY_COL, HOUR_COL]]
    mask_nan = df_unificado_dia["_dia"].isin(lista_para_nan)
    
    df_unificado_dia.loc[mask_nan, cols_para_nan] = np.nan

    # --- D. Limpieza final ---
    columnas_finales = df.columns.tolist()
    df_final = df_unificado_dia[columnas_finales].sort_values(DAY_COL).reset_index(drop=True)

    dfs_23_59_filtrado[hoja] = df_final

print("\n--- Proceso de filtrado a 23:59 completado (CON CREACIÓN DE FILAS) ---")
print(f"Hojas procesadas y guardadas en 'dfs_23_59_filtrado': {len(dfs_23_59_filtrado)}")
print(f"Hojas saltadas: {len(hojas_saltadas)}")

# Verificación de simulación
if "Consolidado EE" in dfs_23_59_filtrado:
    print("\n--- Ejemplo de resultado (Consolidado EE) ---")
    print(dfs_23_59_filtrado["Consolidado EE"].to_string())


Se anularán los datos de 10 días únicos.
Se usará 'keep=first' para 7 días con mala monotonía.
[Consolidado KPI] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Consolidado Produccion] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Produccion] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Consolidado EE] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Energia] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Consolidado Agua] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Agua] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Consolidado GasVapor] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Gas y Vapor] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Consolidado Aire] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores Aire] Creando 8 filas 'NaN' para días que no tenían 23:59...
[Totalizadores CO2] Creando 8 filas 'NaN' para días que n

#### Código para interpolar los datos faltantes

In [42]:
import pandas as pd
import numpy as np

DAY_COL  = "DIA"
HOUR_COL = "HORA"

def interpolar_nans_existentes(df, day_col=DAY_COL, hour_col=HOUR_COL):
    g = df.copy()

    # Convertimos la columna 'DIA' en un datetime real.
    g['_fecha'] = pd.to_datetime(g[day_col], errors='coerce', dayfirst=True)
    
    # Nos aseguramos de que no haya fechas nulas y establecemos esa fecha como el ÍNDICE. Esto es OBLIGATORIO para que method="time" funcione.
    g = g.dropna(subset=['_fecha']).set_index('_fecha')

    # --- 2. Interpolar SOLO columnas numéricas ---
    num_cols = g.select_dtypes(include="number").columns
    
    if len(num_cols) > 0:
        # .interpolate() solo rellena los NaN. Los días con datos (los "buenos") se usan como referencia.
        # method="time" calcula la distancia entre las fechas.
        g[num_cols] = g[num_cols].interpolate(method="time")
    
        # Si el primer o último día era NaN, la interpolación no puede rellenarlos. Usamos ffill (forward-fill) 
        # y bfill (backward-fill) para rellenar esos huecos.
        g[num_cols] = g[num_cols].ffill().bfill()

    # Nos aseguramos de que todas las filas tengan la hora 23:59
    if hour_col in g.columns:
        g[hour_col] = g[hour_col].fillna("23:59:00")

    # Devolvemos el DataFrame a su estado original (sin índice _fecha)
    g = g.reset_index(drop=True) 
    return g

print("Iniciando interpolación de datos...")
dfs_interpolado = {}

# Iteramos sobre el diccionario que ya filtramos a 23:59
for nombre, df in dfs_23_59_filtrado.items():
    if df.empty:
        continue
    print(f"Interpolando hoja: {nombre}...")
    dfs_interpolado[nombre] = interpolar_nans_existentes(df)

print("\n--- Interpolación completada ---")
print(f"Resultados guardados en el diccionario 'dfs_interpolado'.")

# Para verificar, podés imprimir el resultado de la simulación:
if "Consolidado EE" in dfs_interpolado:
    print("\n--- Ejemplo de resultado interpolado (Consolidado EE) ---")
    print(dfs_interpolado["Consolidado EE"].to_string())

#sobre escribir el diccionario original
dfs_2021_2022 = dfs_interpolado

Iniciando interpolación de datos...
Interpolando hoja: Consolidado KPI...
Interpolando hoja: Consolidado Produccion...
Interpolando hoja: Totalizadores Produccion...
Interpolando hoja: Consolidado EE...
Interpolando hoja: Totalizadores Energia...
Interpolando hoja: Consolidado Agua...
Interpolando hoja: Totalizadores Agua...
Interpolando hoja: Consolidado GasVapor...
Interpolando hoja: Totalizadores Gas y Vapor...
Interpolando hoja: Consolidado Aire...
Interpolando hoja: Totalizadores Aire...
Interpolando hoja: Totalizadores CO2...
Interpolando hoja: Totalizadores Efluentes...
Interpolando hoja: Totalizadores Glicol...
Interpolando hoja: Seguimiento Dia...
Interpolando hoja: Auxiliar...

--- Interpolación completada ---
Resultados guardados en el diccionario 'dfs_interpolado'.

--- Ejemplo de resultado interpolado (Consolidado EE) ---
           DIA      HORA   Planta (Kw)  Elaboracion (Kw)   Bodega (Kw)  Cocina (Kw)  Envasado (Kw)  Linea 2 (Kw)  Linea 3 (Kw)  Linea 4 (Kw)  Servicios (

Código para verificar fechas

In [43]:
fecha_buscar = "2022-05-31"

print(f"\n--- Datos para el día {fecha_buscar} ---")
for nombre_hoja, df in dfs_2021_2022.items():
    # Verificar si el DataFrame tiene la columna DIA
    if "DIA" in df.columns:
        # Filtrar por la fecha específica
        datos_dia = df[df["DIA"].dt.strftime("%Y-%m-%d") == fecha_buscar]
        
        if not datos_dia.empty:
            print(f"\nHoja: {nombre_hoja}")
            print(f"Registros encontrados: {len(datos_dia)}")
            print(datos_dia.to_string())
    else:
        print(f"\nLa hoja {nombre_hoja} no tiene columna DIA")


--- Datos para el día 2022-05-31 ---


## Unificación de los diccionarios

In [44]:
DAY_COL  = "DIA"
HOUR_COL = "HORA"

RANGOS = {
    "dfs_21_22": [("2021-01-01", "2021-12-31")],
    "dfs_22_23": [("2022-01-01", "2022-12-31")],
    "dfs_23_24": [("2023-01-01", "2023-12-30"),
                  ("2024-07-01", "2024-10-26")],
}

HOJAS_INCLUIR = [
    "Consolidado KPI", "Consolidado Produccion", "Totalizadores Produccion", "Consolidado EE", "Totalizadores Energia",
    "Consolidado Agua", "Totalizadores Agua", "Consolidado GasVapor", "Totalizadores Gas y Vapor", "Consolidado Aire",
    "Totalizadores Aire", "Totalizadores Efluentes", "Totalizadores Glicol", "Totalizadores CO2"
]

DICS = {
    "dfs_21_22": dfs_2021_2022,
    "dfs_22_23": dfs_2022_2023,
    "dfs_23_24": dfs_2023_2024,
}

def slice_por_fecha(df, start, end, day_col=DAY_COL):
    if df.empty or day_col not in df.columns:
        return df.iloc[0:0]
    fechas = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True).dt.normalize()
    mask = fechas.between(pd.to_datetime(start), pd.to_datetime(end), inclusive="both")
    return df.loc[mask].copy()

def ordenar_crono(df, day_col=DAY_COL, hour_col=HOUR_COL):
    if df.empty:
        return df
    dia = pd.to_datetime(df[day_col], errors="coerce", dayfirst=True)
    if hour_col in df.columns:
        dt = pd.to_datetime(dia.dt.date.astype(str) + " " + df[hour_col].astype(str),
                            errors="coerce", dayfirst=True)
    else:
        dt = dia
    return (df.assign(_dt=dt)
              .sort_values("_dt", kind="stable", na_position="last")
              .drop(columns="_dt").reset_index(drop=True))


partes_por_hoja = defaultdict(list)

for nombre_dic, dic in DICS.items():
    rangos = RANGOS.get(nombre_dic, [])
    for (inicio, fin) in rangos:
        for hoja, df in dic.items():
            if HOJAS_INCLUIR and hoja not in HOJAS_INCLUIR:
                continue
            recorte = slice_por_fecha(df, inicio, fin)
            if not recorte.empty:
                partes_por_hoja[hoja].append(recorte)

dfs_completo = {}
for hoja, partes in partes_por_hoja.items():
    # Unificar columnas: las faltantes quedan como NaN
    todas_cols = list(set().union(*(p.columns for p in partes)))
    partes_alineadas = [p.reindex(columns=todas_cols) for p in partes]
    combinado = pd.concat(partes_alineadas, ignore_index=True, sort=False)

    # Orden temporal final
    if DAY_COL in combinado.columns:
        combinado = ordenar_crono(combinado, DAY_COL, HOUR_COL)
    dfs_completo[hoja] = combinado

In [45]:
dfs_completo['Consolidado Produccion'] = dfs_completo['Consolidado Produccion'].drop(columns="Fecha/Hora", errors="ignore")
dfs_completo['Consolidado EE'] = dfs_completo['Consolidado EE'].drop(columns=['Fecha/Hora', 'Kw de Frio'], errors="ignore")

In [46]:
PATRONES_DIA = ("dia", "Dia", "DIA")

def detectar_col_dia(df, patrones=PATRONES_DIA):
    cols = [str(c) for c in df.columns]
    cand = [c for c in cols if any(p in c.lower() for p in patrones)]
    if not cand:
        raise ValueError("No se encontró columna de día/fecha en un DF.")
    # Heurística: prioriza nombres más específicos
    preferencia = ["dia", "día", "Dia", "DIA"]
    cand_orden = sorted(cand, key=lambda c: next((i for i,p in enumerate(preferencia) if p in c.lower()), 99))
    return cand_orden[0]

def normalizar_dia_col(df, col_dia):
    out = df.copy(deep=True)
    out[col_dia] = pd.to_datetime(out[col_dia], errors="coerce")
    # Si trae hora, nos quedamos con la fecha (día civil)
    out[col_dia] = out[col_dia].dt.normalize()
    # Renombramos a un nombre canónico común
    if col_dia != "dia":
        out = out.rename(columns={col_dia: "dia"})
    return out

def deduplicar_por_dia(df):
    # Si hay múltiples filas por día en un DF, evitamos explosiones en los merges
    # Estrategia simple: nos quedamos con la ultima por día (ajusta si necesitas otra agregación)
    if df.duplicated("dia").any():
        df = df.sort_values("dia").drop_duplicates("dia", keep="last")
    return df

def mergear_por_dia(dfs_completo):
    dfs_norm = []
    for k, df in dfs_completo.items():
        col = detectar_col_dia(df)
        tmp = normalizar_dia_col(df, col)
        tmp = deduplicar_por_dia(tmp)
        # Evita choques de nombres: agrega sufijo con la clave del dict a las columnas no-clave
        cols_no_clave = [c for c in tmp.columns if c != "dia"]
        tmp = tmp[["dia"] + cols_no_clave].add_suffix(f"__{k}")
        tmp = tmp.rename(columns={f"dia__{k}": "dia"})
        dfs_norm.append(tmp)

    # Merge iterativo (outer) por 'dia'
    from functools import reduce
    df_unificado = reduce(lambda l, r: pd.merge(l, r, on="dia", how="outer"), dfs_norm)

    # Orden final
    df_unificado = df_unificado.sort_values("dia").reset_index(drop=True)
    return df_unificado

df_unificado = mergear_por_dia(dfs_completo)

In [47]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Columns: 425 entries, dia to Tot Fermantacion_Cocina__Totalizadores Glicol
dtypes: datetime64[ns](2), float64(401), object(22)
memory usage: 3.9+ MB


In [48]:
df_unificado.select_dtypes(include=["object"]).columns.tolist()

['HORA__Consolidado KPI',
 'Unnamed: 124__Consolidado KPI',
 'HORA__Consolidado Produccion',
 'HORA__Totalizadores Produccion',
 'Nivel Silo Bagazo Norte (1)__Totalizadores Produccion',
 'HORA__Consolidado EE',
 'HORA__Totalizadores Energia',
 'KW Trafo 8__Totalizadores Energia',
 'HORA__Consolidado Agua',
 'HORA__Totalizadores Agua',
 'HORA__Consolidado GasVapor',
 'HORA__Totalizadores Gas y Vapor',
 'HORA__Consolidado Aire',
 'HORA__Totalizadores Aire',
 'HORA__Totalizadores CO2',
 'HORA__Totalizadores Efluentes',
 'Totalizador Bba Envasado__Totalizadores Efluentes',
 'Totalizador Bba P2__Totalizadores Efluentes',
 'Totalizador Bba P1__Totalizadores Efluentes',
 'Totalizador Bba P51__Totalizadores Efluentes',
 'Totalizador Bba P4__Totalizadores Efluentes',
 'HORA__Totalizadores Glicol']

In [49]:
cols_float = [
    "Nivel Silo Bagazo Norte (1)__Totalizadores Produccion",
    "Totalizador Bba P51__Totalizadores Efluentes",
    "Totalizador Bba P2__Totalizadores Efluentes",
    "Totalizador Bba P4__Totalizadores Efluentes",
    "Totalizador Bba Envasado__Totalizadores Efluentes",
    "Totalizador Bba P1__Totalizadores Efluentes",
    "KW Trafo 8__Totalizadores Energia",
]

df_unificado[cols_float] = (df_unificado[cols_float].astype(str).apply(lambda s: s.str.replace(r"\.", "", regex=True).str.replace(",", ".", regex=False))
      .apply(pd.to_numeric, errors="coerce").astype("float64")
)

In [50]:
cols_hora = [
    'HORA__Consolidado KPI',
    'HORA__Consolidado Produccion',
    'HORA__Totalizadores Produccion',
    'HORA__Consolidado EE',
    'HORA__Totalizadores Energia',
    'HORA__Consolidado Agua',
    'HORA__Totalizadores Agua',
    'HORA__Consolidado GasVapor',
    'HORA__Totalizadores Gas y Vapor',
    'HORA__Consolidado Aire',
    'HORA__Totalizadores Aire',
    'HORA__Totalizadores CO2',
    'HORA__Totalizadores Efluentes',
    'HORA__Totalizadores Glicol',
]

# 1) Igualdad exacta columna a columna (por fila), tratando NaN como iguales
base = df_unificado[cols_hora[0]].fillna("__NA__")
iguales_mask = df_unificado[cols_hora].fillna("__NA__").eq(base, axis=0)

# 2) ¿Todas las columnas son iguales en todas las filas?
todas_iguales = bool(iguales_mask.all().all())
print("¿Todas las HORA__ son iguales en todas las filas?:", todas_iguales)

# 3) Filas donde NO coinciden todas
filas_ok = iguales_mask.all(axis=1)
diff_rows = df_unificado.loc[~filas_ok, cols_hora]
print("Filas con diferencias:", len(diff_rows))

print(diff_rows.head(10))

¿Todas las HORA__ son iguales en todas las filas?: False
Filas con diferencias: 1213
  HORA__Consolidado KPI HORA__Consolidado Produccion HORA__Totalizadores Produccion HORA__Consolidado EE HORA__Totalizadores Energia HORA__Consolidado Agua HORA__Totalizadores Agua HORA__Consolidado GasVapor HORA__Totalizadores Gas y Vapor HORA__Consolidado Aire HORA__Totalizadores Aire HORA__Totalizadores CO2 HORA__Totalizadores Efluentes HORA__Totalizadores Glicol
0                   NaN                     23:59:00                            NaN                  NaN                         NaN                    NaN                      NaN                        NaN                             NaN                    NaN                      NaN                     NaN                           NaN                        NaN
1              23:59:00                          NaN                       23:59:00             23:59:00                    23:59:00               23:59:00                 23:59

In [51]:
# crea la columna HORA
df_unificado["HORA"] = pd.to_datetime(df_unificado["HORA__Consolidado KPI"], errors="coerce")

# elimina todas las columnas de hora originales
df_unificado = df_unificado.drop(columns=[c for c in cols_hora if c in df_unificado.columns])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4528\3562094624.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_unificado["HORA"] = pd.to_datetime(df_unificado["HORA__Consolidado KPI"], errors="coerce")


In [52]:
df_unificado.to_csv("data/dataset_unificado.csv", index=False)

## Checksum

In [ ]:
# --- Checksum Calculation Block ---
import hashlib
import json
import os

print("\\n--- Calculando Checksum de Datos Crudos ---")

# --- Paso 2: Reunir todos los DataFrames crudos ---
lista_dfs_crudos = []
if 'dfs_2023_2024' in locals():
    lista_dfs_crudos.extend(dfs_2023_2024.values())
if 'dfs_2022_2023' in locals():
    lista_dfs_crudos.extend(dfs_2022_2023.values())
if 'dfs_2021_2022' in locals():
    lista_dfs_crudos.extend(dfs_2021_2022.values())

print(f"Total de hojas (DataFrames) crudos encontrados: {len(lista_dfs_crudos)}")

if not lista_dfs_crudos:
    print("¡Advertencia! No se encontraron DataFrames crudos para calcular el checksum.")
else:
    # --- Paso 3: Unificar todo ---
    # Usamos sort=False para eficiencia, ordenaremos después explícitamente
    df_crudo_total = pd.concat(lista_dfs_crudos, ignore_index=True, sort=False)
    print(f"DataFrame crudo unificado tiene {df_crudo_total.shape[0]} filas y {df_crudo_total.shape[1]} columnas.")

    # --- Paso 4: ¡Ordenar! ---
    # Asumimos que 'HORA' es la columna de timestamp más fiable en los datos crudos
    columna_orden = 'HORA'
    if columna_orden in df_crudo_total.columns:
        print(f"Ordenando datos crudos por '{columna_orden}'...")
        # Aseguramos que HORA sea datetime para un orden correcto, ignorando errores
        df_crudo_total[columna_orden] = pd.to_datetime(df_crudo_total[columna_orden], errors='coerce')
        # Ordenamos, poniendo NaT (fechas no válidas) al final para consistencia
        df_crudo_ordenado = df_crudo_total.sort_values(by=columna_orden, kind='stable', na_position='last').reset_index(drop=True)
    else:
        print(f"¡Advertencia! No se encontró la columna '{columna_orden}' para ordenar. El checksum podría no ser reproducible.")
        # Como fallback MUY BÁSICO, intentamos ordenar por todas las columnas
        # Esto es lento y menos robusto, pero mejor que nada.
        try:
            df_crudo_ordenado = df_crudo_total.sort_values(by=df_crudo_total.columns.tolist()).reset_index(drop=True)
            print("Fallback: Ordenando por todas las columnas.")
        except Exception as e:
            print(f"Error al intentar ordenar por todas las columnas: {e}. Checksum no se calculará.")
            df_crudo_ordenado = None # Marcamos para no continuar

    if df_crudo_ordenado is not None:
        # --- Paso 5: Convertir a bytes ---
        print("Convirtiendo DataFrame ordenado a bytes...")
        try:
            datos_en_bytes = df_crudo_ordenado.to_csv(index=False, encoding='utf-8').encode('utf-8')
        except Exception as e:
            print(f"Error al convertir a CSV/bytes: {e}. Usando representación de string como fallback.")
            # Fallback muy simple si to_csv falla (raro)
            datos_en_bytes = str(df_crudo_ordenado.to_dict(orient='records')).encode('utf-8')


        # --- Paso 6: Calcular Hash MD5 ---
        print("Calculando hash MD5...")
        hash_md5 = hashlib.md5(datos_en_bytes).hexdigest()
        print(f"¡Checksum MD5 de datos crudos!: {hash_md5}")

        # --- Paso 7: Guardar ---
        ruta_checksum = os.path.join('data', 'checksums.json')
        checksum_info = {
            'datos_crudos_unificados_v2': hash_md5 # v2 para diferenciar si ya había uno
        }
        try:
            with open(ruta_checksum, 'w') as f:
                json.dump(checksum_info, f, indent=4)
            print(f"Checksum guardado exitosamente en: {ruta_checksum}")
        except Exception as e:
            print(f"Error al guardar el checksum en {ruta_checksum}: {e}")

# --- Fin del Checksum Block ---

Este df_crudo_total es temporal. Solo existe dentro de ese bloque de código con el único propósito de ser "sellado". No lo usamos para el resto de tu análisis (tú sigues usando tus diccionarios separados, ¡lo cual está perfecto!).

El checksums.json que guardamos en data/ es ese "sello". Cualquiera (¡incluyéndome a mí!) puede descargar tus 3 Excels, correr ese mismo bloque de código, y si el hash que obtenemos es idéntico al que tú guardaste, tenemos 100% de certeza de que estamos trabajando con exactamente los mismos datos crudos.


**Ficha-resumen**
- ¿Qué es df_crudo_total ? Es un DataFrame temporal que une TODOS los datos crudos de TODOS tus archivos y hojas.
- ¿Por qué lo creamos? Para poder generar UN ÚNICO "sello de garantía" (checksum) que represente el 100% de tus datos de origen. Es el "Tesoro Completo".
- ¿Por qué es importante? Permite que cualquier persona (tu colega, tu profesor) verifique con un solo comando si sus archivos Excel son exactamente idénticos a los tuyos. Esto se llama Integridad de Datos y es un pilar de la Reproducibilidad (MLOps).
- ¿Cuál es el paso más importante? df.sort_values(by='HORA') . Sin ordenar los datos, dos personas con los mismos archivos podrían obtener hashes diferentes, y el checksum no serviría para nada.